# Check training

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

In [3]:
root_path = os.path.abspath("..")
if root_path not in sys.path:
    sys.path.append(root_path)
    
os.environ['PRETRAINED_MODELS'] = '../../../pretrained-models.pytorch/'
os.environ['IGNITE_PATH'] = '../../../pytorch-ignite-master/'
ignite_path = os.environ['IGNITE_PATH']
if ignite_path not in sys.path:
    sys.path.append(ignite_path)

In [78]:
CONFIG = {
  "seed": 7777,

  "n_epochs": 200,
  "validate_every_epoch": 4,

  "fold_index": [0, 1, 2, 3, 4, 5, 6],
  "n_splits": 7,

  "batch_size": 32,
  "num_workers": 12,

  "lr_schedulers": [
      {"ExponentialLR": {"gamma": 0.88 } },
      {"ReduceLROnPlateau": {"factor": 0.2, "mode": "max",  "patience": 5, "verbose": "True"}}
  ],
  "early_stopping": {"patience": 10, "mode": "max"},

  "train_aug": {
    "Compose": {
      "transforms": [{
        "RandomChoice": {
          "transforms": [{
            "RandomAffine": {
              "translate": [0.075, 0.075],
              "rotation": [-90, 90],
              "interpolation": 0}}, {
            "RandomFlip": {
              "mode": "h",
              "proba": 0.5}}, {
            "RandomFlip": {
              "mode": "v",
              "proba": 0.5
            }}]}
      }, { "_ToTensor": {}}]}
  },

  "test_aug": {
    "Compose": {
      "transforms": [{ "_ToTensor": {}}]}
  },

  "optimizer": {
    "Adam": {
      "params": [{
        "params": {
          "model.features.parameters": {}},
          "lr": 0.001}, {
        "params": {
          "model.classifier.parameters": {}},
          "lr": 0.001}]}
  }
}

## Setup dataflow

In [40]:
from common.dataflow import get_trainval_batches

In [44]:
train_batches, val_batches = get_trainval_batches(CONFIG['train_aug'], 
                                                  CONFIG['test_aug'], 
                                                  n_splits=CONFIG['n_splits'], 
                                                  fold_index=CONFIG['fold_index'], 
                                                  batch_size=CONFIG['batch_size'], 
                                                  num_workers=CONFIG['num_workers'], 
                                                  seed=CONFIG['seed'])

Check dataflow

In [45]:
for batch_x, batch_y in train_batches:
    break
print(type(batch_x), type(batch_y))
for b in batch_x:
    print(type(b), b.size())

<class 'list'> <class 'torch.cuda.LongTensor'>
<class 'torch.cuda.FloatTensor'> torch.Size([32, 2, 75, 75])
<class 'torch.cuda.DoubleTensor'> torch.Size([32])


In [48]:
batch_y[:10]


 1
 1
 0
 1
 1
 0
 0
 0
 1
 0
[torch.cuda.LongTensor of size 10 (GPU 0)]

## Setup model, loss function and optimizer

In [49]:
from common.torch_common_utils.nn_utils import print_trainable_parameters

In [94]:
from common.models import IcebergSqueezeNet
model = IcebergSqueezeNet(input_n_channels=1).cuda()
# print_trainable_parameters(model)

Check model

In [51]:
import torch
from torch.autograd import Variable

In [52]:
x = Variable(torch.randn((5, 2, 75, 75)).cuda())
y_pred = model(x, None)
type(y_pred), y_pred.size()

(torch.autograd.variable.Variable, torch.Size([5, 2]))

In [53]:
from common.torch_common_utils.training_utils import _apply_variable

In [54]:
for batch_x, batch_y in train_batches:
    batch_x = _apply_variable(batch_x, requires_grad=True)
    batch_y = _apply_variable(batch_y, requires_grad=True)    
    y_pred = model(*batch_x)
    break
type(y_pred), y_pred.size()

(torch.autograd.variable.Variable, torch.Size([32, 2]))

Define loss function

In [55]:
from torch.nn import CrossEntropyLoss
criterion = CrossEntropyLoss().cuda()

In [56]:
for batch_x, batch_y in train_batches:
    batch_x = _apply_variable(batch_x, requires_grad=True)
    batch_y = _apply_variable(batch_y, requires_grad=False)    
    y_pred = model(*batch_x)
    loss = criterion(y_pred.data, batch_y)
    print("Loss : ", loss)
    break

Loss :  Variable containing:
 0.6933
[torch.cuda.FloatTensor of size 1 (GPU 0)]



Setup optimizer

In [57]:
from common.torch_common_utils.deserialization import restore_object, CustomObjectEval

In [58]:
CONFIG['optimizer'] 

{'Adam': {'params': [{'lr': 0.001,
    'params': {'model.features.parameters': {}}},
   {'lr': 0.001, 'params': {'model.classifier.parameters': {}}}]}}

In [59]:
custom_objects = CustomObjectEval(globals=globals())
optimizer = restore_object(CONFIG['optimizer'], custom_objects=custom_objects, verbose_debug=False)

Setup learning rate scheduler

In [72]:
CONFIG['lr_schedulers']

[{'ExponentialLR': {'gamma': 0.88}},
 {'ReduceLROnPlateau': {'factor': 0.2,
   'mode': 'max',
   'patience': 5,
   'verbose': 'True'}}]

In [73]:
params_to_insert = {'optimizer': '_opt' }
custom_objects = {"_opt": optimizer}

lr_schedulers_conf = CONFIG['lr_schedulers']

lr_schedulers = []
if not isinstance(lr_schedulers_conf, (tuple, list)):
    lr_schedulers_conf = [lr_schedulers_conf]
    
for s in lr_schedulers_conf:
    scheduler = restore_object(s, params_to_insert=params_to_insert, custom_objects=custom_objects, verbose_debug=False)
    lr_schedulers.append(scheduler)

In [74]:
lr_schedulers

Early stopping

In [79]:
CONFIG["early_stopping"]

{'mode': 'max', 'patience': 10}

In [95]:
from common.torch_common_utils.training_utils import EarlyStopping

In [96]:
early_stopping = EarlyStopping(**CONFIG['early_stopping'])

### Train with my scripts

In [86]:
from torch.nn.functional import softmax

from common.torch_common_utils.training_utils import train_one_epoch, validate
from common.torch_common_utils.training_utils import write_conf_log, write_csv_log, save_checkpoint, optimizer_to_str
from common.torch_common_utils.training_utils import accuracy


def accuracy_logits(y_logits, y_true):
    y_pred = softmax(y_logits).data
    return accuracy(y_pred, y_true)

In [87]:
optimizer_to_str(optimizer)

'\nOptimizer: Adam\n- Param group: \n\tbetas: (0.9, 0.999)\n\teps: 1e-08\n\tlr: 0.0001293369914320991\n\tweight_decay: 0\n\tinitial_lr: 0.001\n- Param group: \n\tbetas: (0.9, 0.999)\n\teps: 1e-08\n\tlr: 0.0001293369914320991\n\tweight_decay: 0\n\tinitial_lr: 0.001\n'

In [88]:
output_path = os.path.abspath(os.path.join("..", "output"))

In [89]:
from torch.optim.lr_scheduler import _LRScheduler, ReduceLROnPlateau

In [97]:
from datetime import datetime

best_acc = 0
now = datetime.now()

logs_path = os.path.join(output_path, 'logs', 'squeezenet_%s' % now.strftime("%Y%m%d_%H%M"))
if not os.path.exists(logs_path):
    os.makedirs(logs_path)    
    
    
config_str = ""
for k, v in CONFIG.items():
    config_str += "{}: {}\n".format(k, v)
write_conf_log(logs_path, config_str)
    
for epoch in range(0, CONFIG['n_epochs']):
    
    for scheduler in lr_schedulers:
        if isinstance(scheduler, _LRScheduler):
            scheduler.step()

    # train for one epoch
    ret = train_one_epoch(model, train_batches, criterion, optimizer, epoch, 
                          CONFIG['n_epochs'], 
                          avg_metrics=[accuracy_logits, ])
    if ret is None:
        break
    loss, acc = ret

    # evaluate on validation set
    if epoch % CONFIG['validate_every_epoch'] == 0:
        ret = validate(model, val_batches, criterion, avg_metrics=[accuracy_logits, ])
        if ret is None:
            break
        val_loss, val_acc = ret
        
        for scheduler in lr_schedulers:
            if isinstance(scheduler, ReduceLROnPlateau):
                scheduler.step(val_acc)
                
        if early_stopping(val_acc):
            break
 
        # remember best accuracy and save checkpoint
        if val_acc > best_acc:
            best_acc = max(val_acc, best_acc)
            save_checkpoint(logs_path, 'val_acc', {
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'val_acc': val_acc,
                'optimizer' : optimizer.state_dict()})

  0%|          | 0/43 [00:00<?, ?it/s]

EarlyStopping: 1 / 10


  0%|          | 0/43 [00:00<?, ?it/s]

EarlyStopping: 2 / 10


  0%|          | 0/43 [00:00<?, ?it/s]

...

EarlyStopping: 8 / 10


  0%|          | 0/43 [00:00<?, ?it/s]

EarlyStopping: 9 / 10


Validation: 100%|##########| 7/7 [00:00<00:00,  7.71it/s, Loss 198.4539 | accuracy_logits 0.473]

EarlyStopping: 10 / 10
EarlyStopping: STOP


### Train with [pytorch/ignite](https://github.com/pytorch/ignite)

In [19]:
from ignite.trainer import Trainer, TrainingEvents

In [16]:
def training_update_function(batch):
    
    (batch_x, batch_a), batch_y = batch    
    model.train()
    optimizer.zero_grad()
    
    batch_x = Variable(batch_x)
    batch_a = Variable(batch_a)
    batch_y = Variable(batch_y)
    
    # compute output
    batch_y_pred = model(batch_x)
    loss = criterion(batch_y_pred, batch_y)
    loss.backward()
    optimizer.step()
    
    return loss.data[0]

def validation_inference_function(batch):
    
    (batch_x, batch_a), batch_y = batch    
    model.eval()
    
    batch_x = Variable(batch_x, volatile=True)
    batch_a = Variable(batch_a, volatile=True)
    batch_y = Variable(batch_y, volatile=True)
    
    batch_y_pred = model(batch_x)
    loss = criterion(batch_y_pred, batch_y)

    return loss.data[0]


In [17]:
import logging
logger = logging.getLogger('ignite')
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

In [18]:
trainer = Trainer(training_data=train_batches, 
                  validation_data=val_batches, 
                  training_update_function=training_update_function, 
                  validation_inference_function=validation_inference_function)

In [20]:
def display_training_metrics(trainer, *args, **kwargs):
    last_loss = trainer.training_history[-1]
    trainer._logger.info("Loss {}".format(last_loss))    

trainer.add_event_handler(TrainingEvents.TRAINING_EPOCH_COMPLETED, display_training_metrics)

In [21]:
trainer.run(max_epochs=CONFIG['n_epochs'], validate_every_epoch=2)

Training starting with params max_epochs=10 validate_every_epoch=2
INFO:ignite.trainer.Trainer:Training starting with params max_epochs=10 validate_every_epoch=2
Epoch[0] Complete. Time taken: 00:00:01
INFO:ignite.trainer.Trainer:Epoch[0] Complete. Time taken: 00:00:01
Loss 0.691881000995636
INFO:ignite.trainer.Trainer:Loss 0.691881000995636
Validation Complete. Time taken: 00:00:00
INFO:ignite.trainer.Trainer:Validation Complete. Time taken: 00:00:00
Epoch[1] Complete. Time taken: 00:00:01
INFO:ignite.trainer.Trainer:Epoch[1] Complete. Time taken: 00:00:01
Loss 0.6950010657310486
INFO:ignite.trainer.Trainer:Loss 0.6950010657310486
Validation Complete. Time taken: 00:00:00
INFO:ignite.trainer.Trainer:Validation Complete. Time taken: 00:00:00
Epoch[2] Complete. Time taken: 00:00:01
INFO:ignite.trainer.Trainer:Epoch[2] Complete. Time taken: 00:00:01
Loss 0.6961565613746643
INFO:ignite.trainer.Trainer:Loss 0.6961565613746643
Validation Complete. Time taken: 00:00:00
INFO:ignite.trainer.Tr

In [1]:
!cd ../../../pytorch-ignite-master/ && python3 setup.py install

/usr/local/lib/python3.5/dist-packages/setuptools/dist.py:349: UserWarning: Normalizing '0.1.0-alpha' to '0.1.0a0'
  normalized_version,
running install
running bdist_egg
running egg_info
writing ignite.egg-info/PKG-INFO
writing dependency_links to ignite.egg-info/dependency_links.txt
writing top-level names to ignite.egg-info/top_level.txt
writing requirements to ignite.egg-info/requires.txt
reading manifest file 'ignite.egg-info/SOURCES.txt'
writing manifest file 'ignite.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/ignite
copying build/lib/ignite/__init__.py -> build/bdist.linux-x86_64/egg/ignite
copying build/lib/ignite/trainer.py -> build/bdist.linux-x86_64/egg/ignite
byte-compiling build/bdist.linux-x86_64/egg/ignite/__init__.py to __init__.cpython-35.pyc
byte-compiling build/bdist.linux-x86_64/egg/ignite/trainer.py to trainer.cpython-35

In [16]:
import json

In [17]:
with open('test_config.json', 'w') as handler:
    json.dump(CONFIG, handler)

In [13]:
def read_config(filepath):
    with open(filepath, 'r') as handler:
        config = json.load(handler)
    return config

In [18]:
!ls

check_dataflow.ipynb  data_visualization.ipynb
check_training.ipynb  test_config.json


In [26]:
config = read_config("../scripts/train_configs/config_squeezenet_20171129_2155.json")

In [19]:
config = read_config("test_config.json")

In [21]:
config == CONFIG

True

In [25]:
config_str = ""
for k, v in config.items():
    config_str += "{}: {}\n".format(k, v)
config_str

'validate_every_epoch: 5\nseed: 7777\nnum_workers: 12\nn_splits: 10\nlr_scheduler: {\n        "ExponentialLR": {            \n            "gamma": 0.88\n        }\n    }\n    \nn_epochs: 200\ntrain_aug: {\n                    "Compose":\n                        {"transforms": [{"RandomChoice": {"transforms": [\n                                            {"RandomAffine": {"translate": [0.075, 0.075],\n                                                              "rotation": [-90, 90],                                                \n                                                              "interpolation": 0}},\n                                            {"RandomFlip": {"mode": "h", "proba": 0.5}},\n                                            {"RandomFlip": {"mode": "v", "proba": 0.5}}]}},\n                                        {"_ToTensor": {}}\n                                        ]\n                        }\n                    }\n                \ntest_aug: {\n          